In [426]:
import numpy as np
import pandas as pd
import math
from tensorflow import keras
import tensorflow as tf

In [427]:
# Total RIS elements M:
M = 16
sigma = -55
# Coordinates of AP(x_ap, y_ap, z_ap):
AP = [0, 0, 0]

# Coordinates of RIS(x_r, y_r, z_r):
RIS = [300, 0, 0]

# Coordinates of user k(x_k, y_k, z_k):
users = []
for user in range(4):
    users.append([np.random.randint(-500, 800), np.random.randint(-500,500), np.random.randint(-500,500)])

def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)

distance(AP, RIS), distance(AP, users[0]), distance(RIS, users[0])

(300.0, 218.36208462093413, 204.1617006198763)

In [428]:
# Calculating NLos vectors for v, gk:
v_nlos = np.random.normal(0, math.sqrt(0.5), size=(M,1)) + np.random.normal(0, math.sqrt(0.5), size=(M,1))
gk_nlos = np.random.normal(0, math.sqrt(0.5), size=(M,1)) + np.random.normal(0, math.sqrt(0.5), size=(M,1))
rk_nlos = np.random.normal(0, math.sqrt(0.5), size=(1,1)) + np.random.normal(0, math.sqrt(0.5), size=(1,1))


v_nlos = v_nlos.ravel().tolist()
gk_nlos = gk_nlos.ravel().tolist()
rk_nlos = rk_nlos.ravel().tolist()

In [429]:
# Generating v_los:
v_los = []

def sin_a_AR(x1,x2,y1,y2):
    return (y1-y2)/math.sqrt((x1-x2)**2 + (y1-y2)**2)

for m in range(1, M+1):
    v_los.append( math.cos((m-1)*math.pi*sin_a_AR(RIS[0], AP[0], RIS[1], AP[1])) 
              + (1j)*math.sin((m-1)*math.pi*sin_a_AR(RIS[0], AP[0], RIS[1], AP[1])))

In [430]:
# Generating gk_los:
gk_los = [[] for _ in range(4)]

for user in range(4):
    for m in range(1, M+1):
        gk_los[user].append( math.cos((m-1)*math.pi*sin_a_AR(users[user][0], RIS[0], users[user][1], RIS[1])) 
                         + (1j)*math.sin((m-1)*math.pi*sin_a_AR(users[user][0], RIS[0], users[user][1], RIS[1])))

In [431]:
# Generating v[n]:
v = []
rho_0 = 30       # actual rho_0 = -30 (using 1j later)
d_0 = 1
alpha = 3*(10**(-4))
K_AR = 3
K_RU = 3

def PL(d):
    # print(rho_0*((d/d_0)**alpha))
    return rho_0*((d/d_0)**alpha)

def generate_v():
    for m in range(1, M+1):
        v.append( 1j*math.sqrt(PL(300)) * (math.sqrt(K_AR/(K_AR+1)) *v_los[m-1] + math.sqrt(1/(K_AR+1)) *v_nlos[m-1]) )
    return v

In [432]:
# Generating g_k[n]:
gk = [[] for _ in range(4)]

def generate_gk():
    for user in range(4):
        for m in range(1, M+1):
            gk[user].append( 1j*math.sqrt(PL(distance(RIS, users[user]))) * (math.sqrt(K_RU/(K_RU+1)) *gk_los[user][m-1] + math.sqrt(1/(K_RU+1)) *gk_nlos[m-1]) )
    return gk   

In [433]:
rk = [[] for _ in range(4)]

def generate_rk():
    for user in range(4):
        rk[user].append( 1j*math.sqrt(PL(distance( AP, users[user]))) *rk_nlos[0])
    return rk

In [434]:
# Generating theta
theta = []
def generate_theta():
    for m in range(M):
        theta.append(np.random.uniform(0, 2*math.pi, M))
    # theta.ravel().tolist()
    return theta

theta_matrix = generate_theta()
theta = [theta[i][i] for i in range(len(theta))]

In [435]:
gk = generate_gk()
v = generate_v()
rk = generate_rk()
theta_0 = theta
user_power = 6.5
# print(theta_0)
print(len(rk[0]))
# print(len(theta_0))
# print(len(gk))
# df_u1 = pd.DataFrame(list(zip(gk, v, rk, theta_0)), columns=['gk', 'v', 'rk', 'theta_0'])
dic ={}
for i in range(len(gk[0])):
    dic['gk'+str(i)] = gk[0][i]

for i in range(len(v)):
    dic['v'+str(i)] = v[i]

dic['rk'] = rk[0]

X = pd.DataFrame.from_dict(dic)
# df_u2 = pd.DataFrame(list(zip(gk[1], v, rk[1], theta_0)), columns=['gk', 'v', 'rk', 'theta_0'])
# df_u3 = pd.DataFrame(list(zip(gk[2], v, rk[2], theta_0)), columns=['gk', 'v', 'rk', 'theta_0'])
# df_u4 = pd.DataFrame(list(zip(gk[3], v, rk[3], theta_0)), columns=['gk', 'v', 'rk', 'theta_0'])

# df_u1.shape, df_u2.shape, df_u3.shape, df_u4.shape
y_dic = {}
for i in range(len(theta_0)):
    y_dic['theta_0'+str(i)] = [theta_0[i]]
y_dic['pk'] = [user_power]
print(y_dic)
y = pd.DataFrame.from_dict(y_dic)
print(X.shape, y.shape)

1
{'theta_00': [2.164743148001766], 'theta_01': [3.5899281001462815], 'theta_02': [2.4795880089067386], 'theta_03': [2.8790604345430713], 'theta_04': [3.0080185005079616], 'theta_05': [3.043760718752964], 'theta_06': [0.4143735909481563], 'theta_07': [3.291685212676275], 'theta_08': [4.482742967466999], 'theta_09': [3.639053064442328], 'theta_010': [4.041607167867285], 'theta_011': [5.087303158095529], 'theta_012': [6.197515886809353], 'theta_013': [1.6231698133278845], 'theta_014': [1.3685569014985868], 'theta_015': [1.644970749642008], 'pk': [6.5]}
(1, 33) (1, 17)


In [455]:
class NeuralNetwork:
    def __init__(self):
        self.model = keras.models.Sequential()
        self.gk = X[[col for col in X if col.startswith('gk')]].to_numpy()
        self.v = X[[col for col in X if col.startswith('v')]].to_numpy()
        self.rk = X[[col for col in X if col.startswith('rk')]].to_numpy()

    def dot_product(self, a_vector,b_vector):
    #a1 x b1 + a2 * b2..an*bn return scalar
        a_vector= tf.cast(a_vector, tf.complex64)
        b_vector= tf.cast(b_vector, tf.complex64)
        return sum([an*bn for an,bn in zip(a_vector,b_vector)])

    def custom_loss_function(self, y_true, y_pred):
        squared_difference = tf.square(y_true - y_pred)
        value = tf.reduce_mean(squared_difference, axis=-1)
        print(type(value))
        print(value.shape)
        print(value)
        return value

    def loss_func(self, y_true, y_pred):
        # print(type(y_pred))
        # tf.print(tf.shape(y_pred[:,:16]))
        # tf.print(tf.shape(y_pred))
        # y_true = y_true*0
        # print(y_true)
        # print(y_pred)
        theta_1 = tf.linalg.diag(y_pred[:,:16][0])
        # pk = y_pred[:,-1]
        # print(theta_1)
        # pk_ = pk[0]
        # print(pk_)
        v_ = tf.transpose(tf.convert_to_tensor(self.v, dtype=tf.complex128))
        # print(v_)
        a = tf.math.conj(tf.convert_to_tensor(self.gk))
        # print(a)
        theta_1= tf.cast(theta_1, tf.complex128)
        b = tf.tensordot(theta_1, v_, axes=1)
        b_ = tf.convert_to_tensor(b)
        # print(b_)
        rk_ = tf.convert_to_tensor(self.rk[0], dtype=tf.complex128)
        # print(rk_)
        c = tf.tensordot(a, b_, axes=1)
        c_ = tf.convert_to_tensor(c[0])
        # print(c_)
        # hk = (tf.linalg.norm( tf.convert_to_tensor(rk) + np.tensordot(a, b)))**2
        # print(type(c_))
        # print(type(rk_))
        hk = (tf.linalg.norm(rk_ + c_))**2
        # print(hk)
        hk_ = tf.fill((1,), hk)
        # print(hk_)
        return hk_
        # RkO = 1/4*(np.log2( 1+ ((pk[0] * hk) * 4)/sigma**2 )) #4 -> users
        # rate = (pk*hk)
    # def loss_func(self, y_true, y_pred):
    #     tf.print(y_pred)
    #     squared_difference = tf.square(y_true - y_pred)
    #     return tf.reduce_mean(squared_difference, axis=-1)

    def InputLayer(self, Input):
        self.model.add(keras.layers.Flatten(input_shape=Input))
    def AddLayers(self, n_layers, neurons, activation):
        for i in range(n_layers):
            self.model.add(keras.layers.BatchNormalization())
            self.model.add(keras.layers.Dense(neurons[i], activation=activation[i],kernel_initializer='he_normal'))
            # self.model.add(keras.layers.Dropout(0.2))
            # print(self.model.summary())
    def fit(self,loss, X_train, y_train):
        # self.loss_func(X_train, y_train)
        optimizer = keras.optimizers.Adam(learning_rate=0.0003)
        self.model.compile(loss=self.loss_func, optimizer=optimizer, run_eagerly=True)
        self.model.fit(X_train, y_train, epochs=10)
    
    def predict(self, X):
        y = self.model.predict(X)
        return y

In [456]:
nmodel = NeuralNetwork()
nmodel.InputLayer([X.shape[1]])
nmodel.AddLayers(2,[30,y.shape[1]], ["relu","tanh"])
nmodel.fit("mse", X,y)
ypred = nmodel.predict(X)
print("power allocation = ",ypred[0][16])
print("theta[n] = ", ypred[0][:16])

Epoch 1/10
1/1 [==============================] - 0s 117ms/step - loss: 2.3435
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 1.8175
Epoch 3/10
1/1 [==============================] - 0s 18ms/step - loss: 1.3585
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 0.9678
Epoch 5/10
1/1 [==============================] - 0s 23ms/step - loss: 0.6461
Epoch 6/10
1/1 [==============================] - 0s 18ms/step - loss: 0.3932
Epoch 7/10
1/1 [==============================] - 0s 20ms/step - loss: 0.2071
Epoch 8/10
1/1 [==============================] - 0s 20ms/step - loss: 0.0841
Epoch 9/10
1/1 [==============================] - 0s 20ms/step - loss: 0.0181
Epoch 10/10
1/1 [==============================] - 0s 22ms/step - loss: 9.5069e-05
power allocation =  0.9980768
theta[n] =  [ 0.75001377 -0.96453804  0.99819076 -0.1327869  -0.97391087 -0.9981955
 -0.8820091   0.9956111  -0.7964004   0.99850786  0.98888445 -0.12552495
 -0.9940127  -0.92905945 -0

In [438]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, BatchNormalization, LSTM

# model = Sequential([
#     Dense(64, input_shape=(4,), activation="relu"),
#     LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1),
#     Dense(128, activation='relu'),
#     Dense(128, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(64, activation='relu'),
#     BatchNormalization(),
#     Dense(3, activation='tanh')
# ]);

# e = 32
# N = 16

# for epoch in range(e):
#     # Reset the environment and initial state
#     gk = generate_gk()
#     v = generate_v()
#     rk = generate_rk()
#     theta = generate_theta()

#     for n in range(N):
#     # Input the current phase shifts Θ[n − 1] and the CSI {gk[n], v[n], rk[n]} to the DNN
